## Day 16: Chronal Classification

https://adventofcode.com/2018/day/16

### Part 1

In [32]:
from pyrsistent import pvector, pmap, pset
import operator as op
from parse import parse


def opr(f):
    def fr(registers, a, b, c):
        return registers.set(c, f(registers[a], registers[b]))
    
    return fr


def opi(f):
    def fi(registers, a, b, c):
        return registers.set(c, f(registers[a], b))
    
    return fi


addr = opr(op.add)
addi = opi(op.add)
mulr = opr(op.mul)
muli = opi(op.mul)

banr = opr(op.and_)
bani = opi(op.and_)
borr = opr(op.or_)
bori = opi(op.or_)

def setr(r, a, b, c):
    return r.set(c, r[a])
def seti(r, a, b, c):
    return r.set(c, a)

def gtir(r, a, b, c):
    return r.set(c, 1 if a > r[b] else 0)
def gtri(r, a, b, c):
    return r.set(c, 1 if r[a] > b else 0)
def gtrr(r, a, b, c):
    return r.set(c, 1 if r[a] > r[b] else 0)

def eqir(r, a, b, c):
    return r.set(c, 1 if a == r[b] else 0)
def eqri(r, a, b, c):
    return r.set(c, 1 if r[a] == b else 0)
def eqrr(r, a, b, c):
    return r.set(c, 1 if r[a] == r[b] else 0)


opcodes = pvector([addr, addi, mulr, muli, banr, bani, 
                   borr, bori, setr, seti, gtir, gtri,
                   gtrr, eqir, eqri, eqrr])


def matches(before, after, a, b, c, opcode):
    try:
        return opcode(before, a, b, c) == after
    except:
        return False

def num_matches(before, after, a, b, c):
    return sum(1 if matches(before, after, a, b, c, opcode) else 0
               for opcode in opcodes)


assert num_matches(pvector([3, 2, 1, 1]), 
                   pvector([3, 2, 2, 1]), 2, 1, 2) == 3

In [33]:
lines = open('input_1', 'r').readlines()

n = 0

for i in range(0, len(lines), 4):
    before = pvector(parse('Before: [{:d}, {:d}, {:d}, {:d}]', lines[i]).fixed)
    _, a, b, c = parse('{:d} {:d} {:d} {:d}', lines[i + 1]).fixed
    after = pvector(parse('After:  [{:d}, {:d}, {:d}, {:d}]', lines[i + 2]).fixed)
    if num_matches(before, after, a, b, c) >= 3:
        n += 1
        
n

612

### Part 2

Let's see if each operation appears as the only possibility in at least one example.

In [34]:
def op_matches(before, after, a, b, c):
    return [opcode for opcode in opcodes 
            if opcode(before, a, b, c) == after]

op_index = pmap()

for i in range(0, len(lines), 4):
    before = pvector(parse('Before: [{:d}, {:d}, {:d}, {:d}]', lines[i]).fixed)
    op_i, a, b, c = parse('{:d} {:d} {:d} {:d}', lines[i + 1]).fixed
    after = pvector(parse('After:  [{:d}, {:d}, {:d}, {:d}]', lines[i + 2]).fixed)
    matches = op_matches(before, after, a, b, c)
    if len(matches) == 1:
        op_index = op_index.set(op_i, matches[0])

op_index

pmap({4: <function eqrr at 0x7f2ffc04c488>})

No. How many of each are there as possibilities for each op index?

In [56]:
from collections import defaultdict

op_index_poss = defaultdict(pset)

for i in range(0, len(lines), 4):
    before = pvector(parse('Before: [{:d}, {:d}, {:d}, {:d}]', lines[i]).fixed)
    op_i, a, b, c = parse('{:d} {:d} {:d} {:d}', lines[i + 1]).fixed
    after = pvector(parse('After:  [{:d}, {:d}, {:d}, {:d}]', lines[i + 2]).fixed)
    op_index_poss[op_i] = op_index_poss[op_i] | pset(op_matches(before, after, a, b, c))

op_index_poss

defaultdict(<function pyrsistent._pset.pset>,
            {0: pset([<function opr.<locals>.fr at 0x7f2feeb11e18>, <function opi.<locals>.fi at 0x7f2feeb11048>, <function eqir at 0x7f2feeb11f28>, <function gtri at 0x7f2feeb11950>, <function opi.<locals>.fi at 0x7f2feeb11598>, <function setr at 0x7f2feeb117b8>, <function seti at 0x7f2feeb119d8>, <function opr.<locals>.fr at 0x7f2feeb11bf8>]),
             1: pset([<function gtir at 0x7f2feeb11620>, <function eqrr at 0x7f2ffc04c488>, <function opi.<locals>.fi at 0x7f2feeb116a8>, <function eqri at 0x7f2ffc04cf28>, <function eqir at 0x7f2feeb11f28>, <function gtri at 0x7f2feeb11950>, <function gtrr at 0x7f2feeb11378>, <function opr.<locals>.fr at 0x7f2feeb11bf8>]),
             2: pset([<function gtir at 0x7f2feeb11620>, <function eqrr at 0x7f2ffc04c488>, <function eqri at 0x7f2ffc04cf28>, <function eqir at 0x7f2feeb11f28>, <function gtri at 0x7f2feeb11950>, <function gtrr at 0x7f2feeb11378>]),
             3: pset([<function opr.<locals>.f

Ok, it looks like we might be able to work one by one. When there is only one possibility, assign that and remove it from the other possibilities. Repeat until hopefully all opcodes are assigned.

In [57]:
op_index_poss = defaultdict(pset)

for i in range(0, len(lines), 4):
    before = pvector(parse('Before: [{:d}, {:d}, {:d}, {:d}]', lines[i]).fixed)
    op_i, a, b, c = parse('{:d} {:d} {:d} {:d}', lines[i + 1]).fixed
    after = pvector(parse('After:  [{:d}, {:d}, {:d}, {:d}]', lines[i + 2]).fixed)
    op_index_poss[op_i] = op_index_poss[op_i] | pset(op_matches(before, after, a, b, c))

op_index = pmap()

found = pset()
while op_index_poss:
    for i in list(op_index_poss.keys()):
        op_index_poss[i] = op_index_poss[i] - found
        if len(op_index_poss[i]) == 1:
            opcode = next(iter(op_index_poss[i]))
            found = found.add(opcode)
            op_index = op_index.set(i, opcode)
            del op_index_poss[i]
            
op_index            

pmap({0: <function opi.<locals>.fi at 0x7f2feeb11598>, 1: <function opi.<locals>.fi at 0x7f2feeb116a8>, 2: <function gtir at 0x7f2feeb11620>, 3: <function opr.<locals>.fr at 0x7f2feeb11e18>, 4: <function eqrr at 0x7f2ffc04c488>, 5: <function opi.<locals>.fi at 0x7f2feeb11048>, 6: <function gtrr at 0x7f2feeb11378>, 7: <function setr at 0x7f2feeb117b8>, 8: <function opi.<locals>.fi at 0x7f2feeb11ae8>, 9: <function seti at 0x7f2feeb119d8>, 10: <function opr.<locals>.fr at 0x7f2feeb11bf8>, 11: <function gtri at 0x7f2feeb11950>, 12: <function eqir at 0x7f2feeb11f28>, 13: <function eqri at 0x7f2ffc04cf28>, 14: <function opr.<locals>.fr at 0x7f2ffc126ea0>, 15: <function opr.<locals>.fr at 0x7f2feeb11840>})

Yes they are. Now run the program.

In [61]:
register = pvector([0, 0, 0, 0])

for line in open('input_2', 'r'):
    op_i, a, b, c = [int(x) for x in line.split()]
    register = op_index[op_i](register, a, b, c)
    
register[0]

485